In [ ]:
# imports
from sliderule import icesat2
import matplotlib.pyplot as plt
import matplotlib
import geopandas

In [ ]:
# initialize client (notebook only processes one granule, so one node is sufficient)
icesat2.init("slideruleearth.io", verbose=True, organization="developers", desired_nodes=1)

In [ ]:
# processing parameters
parms = {
    "poly": sliderule.toregion('grandmesa.geojson')['poly'],
    "t0": '2019-11-14T00:00:00Z',
    "t1": '2019-11-15T00:00:00Z',
    "srt": icesat2.SRT_LAND,
    "len": 100,
    "res": 50,
    "pass_invalid": True, 
    "atl08_class": ["atl08_ground", "atl08_canopy", "atl08_top_of_canopy"],
    "phoreal": {"binsize": 1.0, "geoloc": "mean", "use_abs_h": False, "send_waveform": True}
}

In [ ]:
# atl08 request
atl08 = icesat2.atl08p(parms, asset="nsidc-s3", keep_id=True)

In [ ]:
# print dataframe
atl08

In [ ]:
# create 75th percentile column and separate out ground tracks
atl08['75'] = atl08.apply(lambda row : row["canopy_h_metrics"][icesat2.P['75']], axis = 1)
canopy_gt1l = atl08[atl08['gt'] == icesat2.GT1L]

In [ ]:
# plot canopy height
canopy_gt1l["h_canopy"].plot()

In [ ]:
# cplot 75th percentile
canopy_gt1l['75'].plot()

In [ ]:
# Create Sample Waveform Plots
num_plots = 5
waveform_index = [100, 200, 300, 400, 500]
fig,ax = plt.subplots(num=1, ncols=num_plots, sharey=True, figsize=(12, 6))
for x in range(num_plots):
    ax[x].plot([x for x in range(len(canopy_gt1l['waveform'][waveform_index[x]]))], canopy_gt1l['waveform'][waveform_index[x]], zorder=1, linewidth=1.0, color='mediumseagreen')
plt.show()

In [ ]:
# atl06 request
atl06 = icesat2.atl06p(parms, asset="nsidc-s3", keep_id=True)

In [ ]:
# merge dataframes
gdf = geopandas.pd.merge(atl08, atl06, on='extent_id', how='left', suffixes=('.atl08','.atl06')).set_axis(atl08.index)
gdf